In [76]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import NullFormatter
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
import itertools

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.linear_model import ElasticNet
from sklearn import linear_model
from sklearn.linear_model import Ridge

import matplotlib.ticker as ticker
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.feature_selection import mutual_info_classif
from sklearn import preprocessing

In [77]:
df=pd.read_csv('../input/heartcsv/heart.csv')

****Veri denetimi****

In [78]:
df.isna().sum()

In [79]:
df.duplicated().sum()

In [80]:
print(df.info())

**Object tipinde ki özelliklerin algoritmalar tarafından kullanılabilmesi için dönüştürülmesi gerekir.**

In [81]:
to_convert=["Sex", "ChestPainType", "RestingECG", "ExerciseAngina","ST_Slope"]
for feature in to_convert:
    replace_with={}
    for num, value in enumerate(df[feature].unique()):
        replace_with[value]=num
    df[feature]=df[feature].replace(replace_with)

In [82]:
df.head()

In [83]:
df.describe()

In [84]:
fig = px.bar( x=["No","Yes"], y=df['HeartDisease'].value_counts().sort_index(), title="Her sınıf için elemanlar",
            labels={"y": "Elements","x": "HeartDisease"})
fig.update_layout(height=400, width=600,)
fig.show()

Sınıflar oldukça dengeli!

In [85]:
fig = px.bar( x=["M","F"], y=df['Sex'].value_counts().sort_index(), title="Elemanların Cinsiyet Dağılımı",
            labels={"y": "Elements","x": "Sex"},height=400, width=600,)
fig.update_layout()
fig.show()

In [86]:
corr_matrix=df.drop(columns="HeartDisease").corr("spearman")
fig=px.imshow(np.tril(corr_matrix),title="Özellikleri arasındaki Spearman korelasyonu",height=400, width=600,)
fig.show()

In [87]:
to_sort=df.corr("spearman")["HeartDisease"].sort_values(ascending=False)
fig = px.bar( x=to_sort.index[1:], y=to_sort.values[1:], 
             title="Özellikler ve HeartDisease arasındaki Spearman korelasyonu",labels={"y": "Correlation","x": "Feature"})
fig.add_hline(y=0)
fig.show()

In [88]:
import plotly.graph_objects as go
for feature in df.columns[:-1]: 
    fig = make_subplots(rows=1, cols=2)
    fig.add_trace(go.Box( y=df[feature]),row=1,col=1)
    fig.add_trace(go.Histogram(x=df[feature]),row=1,col=2)
    fig.update_layout(height=400, width=600, title_text="Boxplot and histogram for feature: "+str(feature))
    fig.update_layout(showlegend=False)
    fig.show()

**Bu parsellerden şunu anlamak mümkündür:**
1-RestingBP, 0 değerine sahip bir aykırı değere sahiptir;
2-Cholesterol, 0 değerine sahip büyük bir gruba (172 element) sahiptir.

İlk durumda, aykırı değer, ortalama ve standart sapma kullanılarak kolayca saptanabilir ve çıkarılabilir. İkinci durum farklı şekilde ele alınmalıdır: tüm bu öğeleri veri kümesinden çıkarmak yanlış olur, bu nedenle bu sorunu ele almanın başka bir yolu, bu değerleri bir regresyon modelinden yapılan tahminlerle değiştirmek olabilir.


***Veri Ön İşleme***


In [89]:
#HeartDisease hariç her özellik için aykırı değer olup olmadığını kontrol eder
for feature in df.columns[:-1]:
    
    mean=df[feature].mean()
    std=df[feature].std()
    #I have chosen to consider outliers all elements outside 6*std
    remove=(df[feature]<mean-6*std) | (df[feature]>mean+6*std)
    if (remove).any():
        print("Removed element(s) with:",feature,round(df[feature].loc[remove].values[0],2))
        df=df.loc[~remove]
        df.reset_index()
print("# of outliers removed:",918-df.shape[0])

Grafiklerden tespit edilen RestingBP aykırı değeri başarıyla tespit edildi ve kaldırıldı! Şimdi yanlış Kolesterol değerlerini değiştirme zamanı.

In [90]:
labels=df["HeartDisease"]
#selection of elements with right values of Cholesterol
filtered_data=df[df["Cholesterol"]>0]
X,y=filtered_data.drop(columns="Cholesterol"),filtered_data["Cholesterol"]
#in the lines of code below we are going to compare regression methods and median/min

#linear regression
lin_reg = LinearRegression().fit(X,y) 
y_pred_lin=lin_reg.predict(X)
lin_error=mean_squared_error(y, y_pred_lin, squared=False)

#mean
mean_error=mean_squared_error(y, [y.mean()]*len(y), squared=False)

#median
median_error=mean_squared_error(y, [y.median()]*len(y), squared=False)

#elastic net
enet = ElasticNet(alpha=0.1, l1_ratio=0.1)
y_pred_enet = enet.fit(X, y).predict(X)
enet_error=mean_squared_error(y, y_pred_enet, squared=False)

#lasso
lasso_reg = linear_model.Lasso(alpha=0.1).fit(X,y)
y_pred_lasso=lasso_reg.predict(X)
lasso_error=mean_squared_error(y, y_pred_lasso, squared=False)

#Ridge
ridge_reg = Ridge(alpha=0.01).fit(X, y)
y_pred_ridge=ridge_reg.predict(X)
ridge_error=mean_squared_error(y, y_pred_ridge, squared=False)

In [91]:
fig = px.bar(x=["Linear","Mean","Median","Elastic Net","Lasso","Ridge"], 
             y=[lin_error,mean_error,median_error,enet_error,lasso_error,ridge_error],
             title="Regression errors",
             labels={"y": "Error","x": "Regression method"})
fig.add_hline(y=min([lin_error,mean_error,median_error,enet_error,lasso_error,ridge_error]))
fig.show()

Tüm regresyon yöntemleri, ikame olarak medyan/ortalama kullanmaktan biraz daha iyi performans gösterir.

Tüm regresyon yöntemleri arasında Linear ve Ridge en düşük hataları elde etti (ancak tüm regresyon hataları arasındaki fark çok küçüktür).

Lineer modelin tahminini kullanacağız.

In [92]:
#sadece Kolesterol değerleri yanlış olan verileri seçmemiz ve Kolesterol sütunundan atmamız gerekiyor
y_pred=lin_reg.predict(df[df["Cholesterol"]==0].drop(columns="Cholesterol"))
#replace the values
df.loc[df["Cholesterol"]==0,"Cholesterol"]=y_pred

In [93]:
scaler = StandardScaler()
labels=df["HeartDisease"]
#splitting in train=80% and test=20%
X_train, X_test, y_train, y_test=train_test_split(df.drop(columns="HeartDisease"),labels,test_size=0.2,random_state=1234)
#standardization of the data in order to ease calculations
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

**SVM Model oluşturma ve değerlendirme**

In [94]:
#tüm sonuçlar bu sözlüğe kaydedilecek
reports_test={}
reports_train={}
#kernels we are going to try
kernels=["linear","rbf","poly","sigmoid"]
#kernels=["rbf"]
for kernel in kernels:
    svm = SVC(kernel=kernel)
    svm.fit(X_train, y_train)
    reports_train[kernel]=classification_report(y_train,svm.predict(X_train),output_dict=True)
    preds=svm.predict(X_test)
    reports_test[kernel]=classification_report(y_test,preds,output_dict=True)

In [95]:
accuracies_train=[round(reports_train[kernel]["accuracy"],2) for kernel in kernels]
recalls_train=accuracies=[round(reports_train[kernel]["1"]["recall"],2) for kernel in kernels]
accuracies_test=[round(reports_test[kernel]["accuracy"],2) for kernel in kernels]
recalls_test=accuracies=[round(reports_test[kernel]["1"]["recall"],2) for kernel in kernels]
fig = make_subplots(rows=1, cols=2, shared_yaxes=True)
traces1=go.Bar(x=kernels, y=accuracies,name="Train")
traces2=go.Bar(x=kernels, y=accuracies,name="Test")
fig.append_trace(go.Bar(x=kernels, y=accuracies_train,name="Train Accuracies"),row=1,col=1)
fig.append_trace(go.Bar(x=kernels, y=recalls_train,name="Train Recalls"),row=1,col=1)
fig.append_trace(go.Bar(x=kernels, y=accuracies_test,name="Test Accuracies"),row=1,col=2)
fig.append_trace(go.Bar(x=kernels, y=recalls_test,name="Test Recalls"),row=1,col=2)
fig.update_layout(title_text='Eğitim ve test veri setinin puanları')
fig.show()

In [96]:
poly_results=pd.DataFrame(columns=['Gamma',"Recall","Accuracy","Precision","AUC"])
rbf_results=pd.DataFrame(columns=['Gamma',"Recall","Accuracy","Precision","AUC"])

#gammas to test
gamma_range=np.arange(0.020,0.2,0.005)
X_train, X_test, y_train, y_test=train_test_split(df.drop(columns="HeartDisease"),labels,test_size=0.2,random_state=1234)
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
for kernel in ["poly","rbf"]:
    for gamma in gamma_range:
        svm = SVC(kernel=kernel,gamma=gamma)
        svm.fit(X_train, y_train)
        preds=svm.predict(X_test)
        results=classification_report(y_test,preds,output_dict=True)
        recall,precision=round(results["1"]["recall"],2),round(results["1"]["precision"],2)
        accuracy=round(results["accuracy"],2)
        fpr, tpr, thresholds = metrics.roc_curve(y_test, preds)
        auc=round(metrics.auc(fpr, tpr),2)
        if (kernel=="poly"):
            poly_results=poly_results.append({'Gamma': gamma,"Recall":recall, "Precision":precision, "Accuracy":accuracy,
                                              "AUC":auc},ignore_index=True)

        else:
            rbf_results=rbf_results.append({'Gamma': gamma,"Recall":recall, "Precision":precision, "Accuracy":accuracy, 
                                             "AUC":auc},ignore_index=True)
        

In [97]:
#we will run a 5-fold validation with gamma=0.035
gamma_results=pd.DataFrame(columns=["Recall","Accuracy","Precision","AUC"])
kf = KFold(n_splits=5, random_state=1234, shuffle=True)
for train_index, test_index in kf.split(X):
    X_train, X_test = df.iloc[train_index].drop(columns="HeartDisease"), df.iloc[test_index].drop(columns="HeartDisease")
    y_train, y_test = labels.iloc[train_index], labels.iloc[test_index]
    X_train=scaler.fit_transform(X_train)
    X_test=scaler.transform(X_test)
    svm = SVC(kernel="poly",gamma=0.035)
    svm.fit(X_train,y_train)
    probs=svm.predict(X_test)
    results=classification_report(y_test,probs,output_dict=True)
    recall,precision=results["1"]["recall"],results["1"]["precision"]
    accuracy=results["accuracy"]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, probs)
    auc=metrics.auc(fpr, tpr)
    gamma_results=gamma_results.append({"Recall":recall, "Precision":precision, "Accuracy":accuracy,
                                              "AUC":auc},ignore_index=True)

In [98]:
fig = px.line(gamma_results, x=gamma_results.index+1, y=["Recall","Precision","Accuracy","AUC"],
             labels={"value": "","x": "Run"},title="5-fold with gamma=0.035")
fig.show()
print("Score stats:")
for feature in gamma_results:
    print(feature+":   "+str(round(gamma_results[feature].mean(),2))+" +/- "+str(round(gamma_results[feature].std(),2)))

**KNN sınıflandırma algoritmasını Kullanaral tahminleme de bulunmak**

In [99]:
df.columns

HeartDisease kolonu hariç diğer kolonlardan oluşan bağımsız değişkenlerden  bir array yaratıyoruz.


In [100]:
X=df[['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope']]
X[0:5]

In [101]:
X=preprocessing.StandardScaler().fit(X).transform(X.astype(float))

HeartDisease featuree kullanarak numpy array yarattım

In [102]:
y=df['HeartDisease'].values
y[0:5]

**KNN gibi noktalar arası mesafe temelli algoritmalarda daha başarılı sonuç elde etmek için veriler normalize edilir.**

In [103]:
X=preprocessing.StandardScaler().fit(X).transform(X.astype(float))
X[0:5]

In [104]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=4)
print('Train set',X_train.shape,y_train.shape)
print('Test set',X_test.shape,y_test.shape)

KNN (K-Nearest Neighbors) Algoritmasını kullanarak sınıflandırma işlemine başlayacağız. İlk olarak k=4 (En yakın 4 komşu içerisindeki yoğunluğa göre karar verecek) olarak modelimizi eğitelim.

In [105]:
from sklearn.neighbors import KNeighborsClassifier
k=4
neigh=KNeighborsClassifier(n_neighbors=k).fit(X_train,y_train)
neigh

neigh ismini verdiğimiz modelimizi eğittik. Şimdi bu model ile prediction yapalım ve sonuçların performansına bakalım.

In [106]:
y_hat=neigh.predict(X_test)
y_hat[0:5]

En optimum k değerini bulmak için modelimizin tüm k değerleri için modelimizi eğitim accuracy score üzerinden bir karşılaştırma yapıp olası en iyi k değeri bulunabilir.
Örnek olsun diye k değerinin 1–10 arasındaki durumlarını inceleyelim.

In [107]:
Ks=10
mean_acc=np.zeros((Ks-1))
std_acc=np.zeros((Ks-1))

ConfustionMx=[];

for n in range(1,Ks):
    neigh=KNeighborsClassifier(n_neighbors=n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1]=metrics.accuracy_score(y_test,yhat)
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])
mean_acc

Elde ettiğimiz sonuçlara grafik üzerinden bakarak optimum K değerini görebiliriz.

In [108]:
plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc-1* std_acc,mean_acc+1 * std_acc,alpha=0.10)
plt.legend(('Accuracy','+/- 3xstd' ))
plt.ylabel('Accuracy')
plt.xlabel('Numbar of Nobors')
plt.tight_layout()
plt.show()

Grafikten görebileceğimiz gibi en iyi değer K=8 olduğunda gerçekleşmiş.

In [111]:
from sklearn.neighbors import KNeighborsClassifier
k=8
neight_8=KNeighborsClassifier(n_neighbors=k,metric='minkowski', p = 2).fit(X_train,y_train)
print('Eğitim verisinin doğruluğu',metrics.accuracy_score(y_train,neight_8.predict(X_train)))
print('Test verisinin doğruluğu',metrics.accuracy_score(y_test,neight_8.predict(X_test)))